# December 12, 2024

https://adventofcode.com/2024/day/12

In [3]:
from collections import defaultdict

In [4]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [5]:
test_str1 = f'''AAAA
BBCD
BBCC
EEEC'''
test_str1 = [ list(x) for x in test_str1.split("\n") ]

test_str2 = f'''OOOOO
OXOXO
OOOOO
OXOXO
OOOOO'''
test_str2 = [ list(x) for x in test_str2.split("\n") ]

test_str3 = f'''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE'''
test_str3 = [list(x) for x in test_str3.split("\n") ]

In [7]:
fn = "../data/2024/12.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [x.strip() for x in text]

# Part 1


In [63]:
class Region:
    def __init__(self, value, adj, locs):
        self.value = value
        
        self.locs = locs

        # keep track of how many adjacencies are in the region
        # 2x2 region has 4
        # 4x1 region has 3
        # This + area is enough to calculate perimeter
        self.adjacencies = adj

        self.area = len(locs)
        self.perimeter = 4*self.area - self.adjacencies
        self.__sides = None

    def sides(self):
        return self.__sides or self.count_sides()


    def count_sides(self):
        # create two dicts, one keyed by row and one keyed by col
        loc_dict = defaultdict(set)
    
        min_r = 9e99
        max_r = 0
        for loc in self.locs:
            r,c = loc
            loc_dict[r].add(c)
            if r < min_r:
                min_r = r
            if r > max_r:
                max_r = r

        loc_dict[min_r-1] = set()
        loc_dict[max_r+1] = set()


        sides = 0
        # we count borders only if its the leftmost for horizontal border
        # or topmost for a vertical border
        for r, cols in loc_dict.items():
            for c in cols:
                # border up, check if it's part of a prev side
                if c not in loc_dict[r-1]:
                    # @ is the current location, [r,c]
                    # count these cases
                    #    xx   Ax    Ax   <-- I think this third option is two sides    
                    #    x@   A@    x@
                    if (c-1 not in loc_dict[r]) or (c-1 in loc_dict[r-1]):
                        sides += 1

                # border down
                #    x@   A@    x@   <-- I think this third option is two sides    
                #    xx   Ax    Ax
                if c not in loc_dict[r+1]:
                    if (c-1 not in loc_dict[r]) or (c-1 in loc_dict[r+1]):
                        sides += 1
                # border left
                #    xx   AA    Ax
                #    x@   A@    A@
                if c-1 not in loc_dict[r]:
                    if (c not in loc_dict[r-1]) or (c-1 in loc_dict[r-1]):
                        sides += 1

                # border right
                #   xx   AA   xA
                #   @x   @x   @x
                if c+1 not in loc_dict[r]:
                    if (c not in loc_dict[r-1]) or (c+1 in loc_dict[r-1]):
                        sides += 1

        self.__sides = sides
        return self.__sides




    def __str__(self):
        return f"Region({self.value}: Size {self.area}, Peri {self.perimeter})"

    def __repr__(self):
        return f"Region({self.value}: Size {self.area}, Peri {self.perimeter})"
        

In [64]:
class Garden:
    def __init__(self, lines):
        self.nr = len(lines)
        self.nc = len(lines[0])
        self.map = [ [x for x in y] for y in lines ]
        # replace letters with blanks when they are explored
        self.unexplored = [ [x for x in y] for y in lines ]
        self.regions = defaultdict(list)

        for r in range(0, self.nr):
            for c in range(0, self.nc):
                if self.unexplored[r][c] != "#":
                    self.__add_region__([r,c])

    
    def __add_region__( self, loc ):
        val = self.unexplored[ loc[0] ][ loc[1] ]
        if val == "#":
            raise BaseException("Location already mapped")
        
        # start a new region
        self.unexplored[ loc[0] ][ loc[1] ] = "#"
        all_locs, adjacencies = self.__map_region__(val, loc )

        new_region = Region( val, adjacencies, all_locs )
        self.regions[val].append(new_region)        
        
        
    def __map_region__( self, val, loc ):
        all_locs = [loc]
        adjacencies = 0

        # iterate over neighbors
        nbrs = self.__get_neighbors__(loc)
        for n in nbrs:
            # Add the adjency always
            if self.map[ n[0] ][ n[1] ] == val:
                adjacencies += 1

            # if it's not explored, also explore it.
            if self.unexplored[ n[0] ][ n[1] ] == val:
                self.unexplored[ n[0] ][ n[1] ] = "#"
                new_locs, new_adj = self.__map_region__(val, n )
                adjacencies += new_adj
                all_locs += new_locs

        return all_locs, adjacencies

    def __get_neighbors__(self, loc):
        r,c = loc
        nbrs = list()
        if r > 0:
            nbrs.append( [r-1, c] )
        if r < self.nr - 1:
            nbrs.append( [r+1, c] )
        if c > 0:
            nbrs.append( [r, c-1] )
        if c < self.nc - 1:
            nbrs.append( [r, c+1] )
        return nbrs

In [65]:
def part1( text ):
    jardin = Garden(text)
    ans = 0
    for val, region_list in jardin.regions.items():
        for reg in region_list:
            ans += reg.area * reg.perimeter

    return ans


In [66]:
part1( test_str1 )

140

In [67]:
part1( test_str2 )

772

In [68]:
part1( test_str3 )

1930

In [69]:
part1( puzz_str )

1471452

# Part 2

In [70]:
test_str4 = f'''EEEEE
EXXXX
EEEEE
EXXXX
EEEEE'''.split("\n")

test_str5 = f'''AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA'''.split("\n")

In [71]:
def part2( text ):
    jardin = Garden(text)
    #return jardin
    ans = 0
    for val, region_list in jardin.regions.items():
        for reg in region_list:
            ans += reg.area * reg.sides()

    return ans


In [73]:
part2(test_str1)

80

In [74]:
part2(test_str4)

236

In [76]:
part2(test_str3)

1206

In [77]:
part2(puzz_str)

863366